In [28]:
import train as t
import importlib
importlib.reload(t)

import tensorflow as tf

import importlib

import numpy as np
from enum import Enum

from functools import reduce

In [31]:
# Create the vocabulary
target_tokens = ["**end**", "**start**", "**unknown**"]

target_tokens.extend(t.get_vocabulary("train"))

token_vocab_size = len(target_tokens)
# todo: document what was lifted from tutorials and what we wrote ourselves
target_token_index = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_target_token_index = dict(
    (i, char) for char, i in target_token_index.items())



print("\n ======================= Loading Data =======================")
#new cell

train_dataset = t.get_data_somehow('train', True, t.hparams['mini_batch_size'], t.hparams['max_token_length'], t.hparams['max_train_num_samples'] , target_token_index)
val_dataset = t.get_data_somehow('val', True, t.hparams['mini_batch_size'], t.hparams['max_token_length'], t.hparams['max_val_num_samples'], target_token_index)

train_encoder_input_data_batches = train_dataset[0]
train_target_texts_batches = train_dataset[1]
train_sequence_lengths_batches = train_dataset[2]
train_decoder_input_data_batches = train_dataset[3]
train_decoder_target_data_batches = train_dataset[4]

val_encoder_input_data_batches = val_dataset[0]
val_target_texts_batches = val_dataset[1]
val_sequence_lengths_batches = val_dataset[2]
val_decoder_input_data_batches = val_dataset[3]
val_decoder_target_data_batches = val_dataset[4]
print("\n ======================= Data Loaded =======================")


HI MOM!  ../data/

 ======================= Loading Data =======================
fresh data coming up
dumped: train
fresh data coming up
dumped: val

 ======================= Data Loaded =======================


In [32]:
tf.reset_default_graph()



g = t.create_graph(token_vocab_size, t.hparams['num_units'], t.hparams['use_attention'], t.hparams['use_encoding_average_as_initial_state'], False)
embedding_decoder = g['embedding_decoder']
decoder_cell = g['decoder_cell']
decoder_initial_state = g['decoder_initial_state']
projection_layer = g['projection_layer']
img = g['img']
decoder_lengths = g['decoder_lengths']
decoder_inputs = g['decoder_inputs']
decoder_outputs = g['decoder_outputs']
train_loss = g['train_loss']


INFO:tensorflow:Summary name conv1/kernel:0/gradient is illegal; using conv1/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv1/bias:0/gradient is illegal; using conv1/bias_0/gradient instead.
INFO:tensorflow:Summary name conv2/kernel:0/gradient is illegal; using conv2/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv2/bias:0/gradient is illegal; using conv2/bias_0/gradient instead.
INFO:tensorflow:Summary name conv3/kernel:0/gradient is illegal; using conv3/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv3/bias:0/gradient is illegal; using conv3/bias_0/gradient instead.
INFO:tensorflow:Summary name conv4/kernel:0/gradient is illegal; using conv4/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv4/bias:0/gradient is illegal; using conv4/bias_0/gradient instead.
INFO:tensorflow:Summary name conv5/kernel:0/gradient is illegal; using conv5/kernel_0/gradient instead.
INFO:tensorflow:Summary name conv5/bias:0/gradient is illegal; using conv5/bias_

In [33]:
len(train_encoder_input_data_batches)

1

In [35]:
sess = tf.Session()
tf_saver = tf.train.Saver(allow_empty=False)
p = '/Users/adamjensen/project-environments/handwriting-to-latex-env/output/checkpoints/model_a_16.ckpt-17'
t.initialize_variables(sess, tf_saver, restore=True, path=p)



restoring
INFO:tensorflow:Restoring parameters from /Users/adamjensen/project-environments/handwriting-to-latex-env/output/checkpoints/model_a_16.ckpt-17


In [36]:
t_loss = t.get_loss(img,
                  train_encoder_input_data_batches,
                  decoder_lengths,
                  train_sequence_lengths_batches,
                  decoder_inputs,
                  train_decoder_input_data_batches,
                  decoder_outputs,
                  train_decoder_target_data_batches,
                  train_loss,
                  sess)

t_loss


158.42501831054688

In [ ]:
v_loss = t.get_loss(img,
                  val_encoder_input_data_batches,
                  decoder_lengths,
                  val_sequence_lengths_batches,
                  decoder_inputs,
                  val_decoder_input_data_batches,
                  decoder_outputs,
                  val_decoder_target_data_batches,
                  train_loss,
                  sess)

v_loss


In [ ]:
# val predictions
batch = val_encoder_input_data_batches[15]

little_variable, logits = t.predict_batch(sess,
                  batch,
                  target_token_index,
                  embedding_decoder,
                  decoder_cell,
                  decoder_initial_state,
                  projection_layer,
                  img)

logits.shape

In [ ]:
# train predictions
batch = train_encoder_input_data_batches[0]
e
little_variable, logits = t.predict_batch(sess,
                  batch,
                  target_token_index,
                  embedding_decoder,
                  decoder_cell,
                  decoder_initial_state,
                  projection_layer,
                  img)

logits.shape

In [ ]:
def get_token_seq(int_sequence):
    
    output_string = ""
    for value in int_sequence:
        output_string += reverse_target_token_index[value] + " "
        if reverse_target_token_index[value] == "**end**":
            break
    return output_string

for idx, seq in enumerate(train_target_texts_batches[0]):
    
    print("Output: ")
    print(get_token_seq(little_variable[idx]))
    print("")
    print("Ground truth: ")
    print(train_target_texts_batches[0][idx])
    print("")